In [1]:
import os
import pandas as pd
import numpy as np

In [8]:
# Define the directory containing CSV files
folder_path = r"D:\Stelco\Work\Dynamic Correlation\Key\Whole_Phase_Good_Clustered_Filtered"
signal_names = [
    'Stand 1 Predicted Run Force',
    'Stand 1 Gap Stick Offset',
    'Tension Reel Calculated Tension',
    'Tension To Gap 2 In Limit',
    'Stand 1-2 Total Tension Feedback',
    'Ramp Greater Than Thread',
    'Stand 3 - Operator Side Force',
    'Flatness Control - Bending In Limit',
    'Stand 1 Run Gap Setpoint',
    'Stand 1 Gap Bite Offset',
    'S1 Operating Bending Trim',
    'Stand 2-3 Tension Reference',
    'Neet Oil Concentration',
    'Morgoil DriveTop Bearing Outflow Temp Stand1',
    'Stand 4 Top Current Feedback',
    'Morgoil DriveTop Bearing Outflow Temp Stand3',
    'Stand 3 Run Gap Setpoint',
    'Stand 2 Total Bending Feedback',
    'Stand 2 Gap Bite Offset',
    'Morgoil OperBottom Bearing Outflow Temp Stand3',
    'Stand 4 Thread Gap Setpoint',
    'X4 Gauge Deviation',
    'Stand 4 DS Total Bending Feedback',
    'Laser 0 Data Valid',
    'Stand 4 - Operator Side Force',
    'Stand 2 Gap Eccentricity Trim',
    'Stand 4 Gap Operator Offset',
    'Stand 3 Total Bending Feedback',
    'Strip In Stand 3',
    'Morgoil OperTop Bearing Outflow Temp Stand1',
    'Strip In Stand 1',
    'X1 Gauge Deviation',
    'Stand 3 Drive Speed Feedback',
    'Stand 2 Gap Thread Offset',
    'Stand 2 Drive Speed Feedback',
    'Stand 1-3 Solution System Pressure',
    'Stand 2 Top Current Feedback',
    'Stand 1-3 Solution Temperature',
    'AGC GE Feedforward Hardness Number',
    'Stand 1 Total Bending Feedback',
    'X0 Gauge Deviation',
    'Stand 3 Bottom Current Feedback',
    'Stand 4 Gap Eccentricity Trim',
    'Stand 2 Gap Stick Offset',
    'Stand 3-4 Tension Reference',
    'Stand 4 Bottom Current Feedback',
    'Stand 1 Bottom Current Feedback',
    'Stand 3 Gap Thread Offset',
    'Stand 2 Bottom Current Feedback',
    'Stand 4 Solution System Pressure',
    'Stand 3 Gap Eccentricity Trim',
    'Stand 4 OS Total Bending Feedback',
    'Stand 1 Gap Thread Offset',
    'AGC Alex Dynamic Feedforward Hardness Number',
    'Stand 3 Top Current Feedback',
    'S2 Operating Bending Trim',
    'Roll Force Hydraulic Tank Level Inches',
    'Roll Force Hydraulics Pressure Feedback',
    'Stand 1 Roll Force Increase Limit (based on predicted run force)',
    'Stand 4 OS Bending Shape Trim',
    'Stand 4 DS Bending Shape Trim'
]

In [19]:
# Initialize list to store coil data
coil_data = []

# Read all CSV files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        df = pd.read_csv(os.path.join(folder_path, filename))
        coil_id = df['STD4_ID'].iloc[0]
        cm_width = df['CM_WIDTH'].iloc[0]
        gauge = df['CP_X4GAUGE'].iloc[0]
        reduction = df['Reduction'].iloc[0]

        # Calculate mean for each signal
        signal_means = {}
        for sig in signal_names:
            if sig in df.columns:
                signal_means[sig] = df[sig].mean()
            else:
                signal_means[sig] = np.nan

        # Append data
        coil_data.append({
            'coil_id': coil_id,
            'CM_WIDTH': cm_width,
            'CP_X4GAUGE': gauge,
            'Reduction':reduction,
            **signal_means
        })

In [20]:
# Convert to DataFrame
coils_df = pd.DataFrame(coil_data)

# Create bins for CM_WIDTH and CP_X4GAUGE
coils_df['Width_Bin'] = pd.cut(coils_df['CM_WIDTH'], bins=np.arange(coils_df['CM_WIDTH'].min(),
                                                                   coils_df['CM_WIDTH'].max() + 10, 10))
coils_df['Gauge_Bin'] = pd.cut(coils_df['CP_X4GAUGE'], bins=np.arange(coils_df['CP_X4GAUGE'].min(),
                                                                     coils_df['CP_X4GAUGE'].max() + 0.02, 0.02))
coils_df['Reduction_Bin'] = pd.cut(coils_df['Reduction'], bins=np.arange(coils_df['Reduction'].min(),
                                                                     coils_df['Reduction'].max() + 10, 10))

# Create a dataframe of unique bucket pairs
unique_buckets = coils_df[['Width_Bin', 'Gauge_Bin','Reduction_Bin']].drop_duplicates().reset_index(drop=True)

# Assign numeric bucket ID
unique_buckets['Bucket_ID'] = range(1, len(unique_buckets) + 1)

# Merge bucket IDs back to coils_df
coils_df = coils_df.merge(unique_buckets, on=['Width_Bin', 'Gauge_Bin','Reduction_Bin'], how='left')

# Create a descriptive bucket name
coils_df['Bucket_Name'] = (
    'Bucket ' + coils_df['Bucket_ID'].astype(str) +
    ': Width ' + coils_df['Width_Bin'].astype(str) +
    ', Gauge ' + coils_df['Gauge_Bin'].astype(str)+
    ', Reduction ' + coils_df['Reduction_Bin'].astype(str)
)

# Generate summary for each bucket
bucket_summary = unique_buckets.merge(
    coils_df.groupby(['Width_Bin', 'Gauge_Bin','Reduction_Bin']).agg({'coil_id': 'count'}).reset_index(),
    on=['Width_Bin', 'Gauge_Bin','Reduction_Bin'], how='left'
).rename(columns={'coil_id': 'Coil_Count'})

bucket_summary['Bucket_Name'] = (
    'Bucket ' + bucket_summary['Bucket_ID'].astype(str) +
    ': Width ' + bucket_summary['Width_Bin'].astype(str) +
    ', Gauge ' + bucket_summary['Gauge_Bin'].astype(str) +
    ', Reduction ' + coils_df['Reduction_Bin'].astype(str)
)

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_10632\1032667989.py:31: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  coils_df.groupby(['Width_Bin', 'Gauge_Bin','Reduction_Bin']).agg({'coil_id': 'count'}).reset_index(),


In [9]:
coil_data = []

# Loop through all CSV files (one per coil)
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        df = pd.read_csv(os.path.join(folder_path, filename))

        coil_id = df['STD4_ID'].iloc[0]
        cm_width = df['CM_WIDTH'].iloc[0]
        gauge = df['CP_X4GAUGE'].iloc[0]
        reduction = df['Reduction'].iloc[0]

        # Ensure 'Phase' column exists in data
        if 'Phase' not in df.columns:
            raise ValueError(f"Missing 'Phase' column in {filename}")

        # Loop over phases present in the CSV (e.g. Phase 1, Phase 2, Phase 3)
        for phase in df['Phase'].unique():
            phase_df = df[df['Phase'] == phase]

            # Calculate signal-wise mean for this phase
            signal_means = {
                sig: phase_df[sig].mean() if sig in phase_df.columns else np.nan
                for sig in signal_names
            }

            # Append one entry per phase
            coil_data.append({
                'coil_id': coil_id,
                'Phase': phase,
                'CM_WIDTH': cm_width,
                'CP_X4GAUGE': gauge,
                'Reduction': reduction,
                **signal_means
            })

# Combine all coils into one dataframe
coils_df = pd.DataFrame(coil_data)

# ------------------------------
# Create bin columns for width, gauge, reduction
# ------------------------------
coils_df['Width_Bin'] = pd.cut(
    coils_df['CM_WIDTH'],
    bins=np.arange(coils_df['CM_WIDTH'].min(), coils_df['CM_WIDTH'].max() + 10, 10)
)
coils_df['Gauge_Bin'] = pd.cut(
    coils_df['CP_X4GAUGE'],
    bins=np.arange(coils_df['CP_X4GAUGE'].min(), coils_df['CP_X4GAUGE'].max() + 0.02, 0.02)
)
coils_df['Reduction_Bin'] = pd.cut(
    coils_df['Reduction'],
    bins=np.arange(coils_df['Reduction'].min(), coils_df['Reduction'].max() + 10, 10)
)

# ------------------------------
# Create unique bucket IDs
# ------------------------------
unique_buckets = coils_df[['Width_Bin', 'Gauge_Bin', 'Reduction_Bin']].drop_duplicates().reset_index(drop=True)
unique_buckets['Bucket_ID'] = range(1, len(unique_buckets) + 1)

# Merge bucket IDs back
coils_df = coils_df.merge(unique_buckets, on=['Width_Bin', 'Gauge_Bin', 'Reduction_Bin'], how='left')

# ------------------------------
# Create descriptive bucket names
# ------------------------------
coils_df['Bucket_Name'] = (
    'Bucket ' + coils_df['Bucket_ID'].astype(str) +
    ': Width ' + coils_df['Width_Bin'].astype(str) +
    ', Gauge ' + coils_df['Gauge_Bin'].astype(str) +
    ', Reduction ' + coils_df['Reduction_Bin'].astype(str)
)

# ------------------------------
# Optional: Bucket summary
# ------------------------------
bucket_summary = (
    coils_df.groupby(['Width_Bin', 'Gauge_Bin', 'Reduction_Bin'])
    .agg(Coil_Count=('coil_id', 'nunique'))
    .reset_index()
    .merge(unique_buckets, on=['Width_Bin', 'Gauge_Bin', 'Reduction_Bin'], how='left')
)

bucket_summary['Bucket_Name'] = (
    'Bucket ' + bucket_summary['Bucket_ID'].astype(str) +
    ': Width ' + bucket_summary['Width_Bin'].astype(str) +
    ', Gauge ' + bucket_summary['Gauge_Bin'].astype(str) +
    ', Reduction ' + bucket_summary['Reduction_Bin'].astype(str)
)

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_7296\3407310347.py:79: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  coils_df.groupby(['Width_Bin', 'Gauge_Bin', 'Reduction_Bin'])


In [10]:
# Save coil data with averages
coils_df.to_csv('bucket_coil_signal_averages_allsignals_phasewise.csv', index=False)

# Save bucket summaries
bucket_summary.to_csv('bucket_properties_allsignals_phasewise.csv', index=False)

print("Processing complete. Files saved: 'coil_signal_averages.csv' and 'bucket_properties.csv'.")

Processing complete. Files saved: 'coil_signal_averages.csv' and 'bucket_properties.csv'.
